# Chargement des données

In [1]:
import pandas as pd

from IPython.display import display
from urllib.parse import quote

doc2start = {"bpt6k63243601": 123, "bpt6k62931221": 151, "bpt6k6286466w": 189, "bpt6k6393838j": 219, "bpt6k6331310g": 216, "bpt6k6292987t": 353, "bpt6k62906378": 288, "bpt6k6391515w": 319, "bpt6k6315927h": 349, "bpt6k6319106t": 324, "bpt6k6315985z": 82, "bpt6k63959929": 82, "bpt6k63197984": 56, "bpt6k6389871r": 77, "bpt6k6319811j": 79, "bpt6k6282019m": 72, "bpt6k6314752k": 190, "bpt6k6305463c": 113, "bpt6k6318531z": 108, "bpt6k6324389h": 72, "bpt6k63243920": 80, "bpt6k6309075f": 96, "bpt6k6333200c": 132, "bpt6k63243905": 134, "bpt6k6333170p": 137, "bpt6k96727875": 135, "bpt6k9764746t": 99, "bpt6k97645375": 123, "bpt6k9672117f": 125, "bpt6k9763554c": 123, "bpt6k9763553z": 105, "bpt6k9677392n": 110, "bpt6k9692809v": 113, "bpt6k9762929c": 129, "bpt6k9672776c": 119, "bpt6k9764647w": 121, "bpt6k9669143t": 145, "bpt6k9677737t": 139, "bpt6k9668037f": 167, "bpt6k96839542": 171, "bpt6k96762564": 185, "bpt6k9685861g": 189, "bpt6k9763471j": 153, "bpt6k9762899p": 157, "bpt6k97630871": 11, "bpt6k9684454n": 235, "bpt6k9732740w": 239, "bpt6k9684013b": 189, "bpt6k9692626p": 305, "bpt6k9685098r": 281, "bpt6k9764402m": 329, "bpt6k97631451": 322, "bpt6k9776121t": 49, "bpt6k9775724t": 33, "bpt6k97774838": 327, "bpt6k9780089g": 339}

def entry2url(row):
    """
    Takes a row of an Annuaire csv and
    transforms it to the corresponding Gallica url
    """
    url = "https://gallica.bnf.fr/ark:/12148/"
    
    directory = row['directory']
    page = row['page'] - doc2start[directory]
    url += f"{row['directory']}/f{row['page']-doc2start[row['directory']]}"
    
    r_strings = []
    if 'name' in row and pd.notna(row['name']):
        r_strings.append(quote(row['name'].replace('.', ' ')))
    if 'job' in row and pd.notna(row['job']):
        r_strings.append(quote(row['job'].replace('.', ' ')))
    if 'street' in row and pd.notna(row['street']):
        r_strings.append(quote(row['street'].replace('.', ' ')))
    if 'number' in row and pd.notna(row['number']):
        r_strings.append(quote(row['number'].replace('.', ' ')))
    
    if len(r_strings) > 0:
        url += f".item.r={'%20'.join(r_strings)}.zoom"
    
    return url

def add_clickable_url(bottin_dataframe):
    bottin_dataframe = bottin_dataframe.copy()
    bottin_dataframe['url'] = bottin_dataframe.apply(entry2url, axis=1)
    #def make_clickable(val):
    #    return '<a href="{}">gallica url</a>'.format(val,val)

    #return bottin_dataframe.style.format(make_clickable, subset=['url'])
    return bottin_dataframe

In [2]:
data_bottin = pd.read_csv('bottin_data_groupe_6.csv')

# Clean

Nettoyer le numéro de rue: garder seulement le premier nombre et éventuellement 'bis'

In [3]:
data_bottin['number_clean'] = data_bottin['number'].str.extract('(^\d+(?: ?bis)?).*')

Drop duplicated entries

In [4]:
data_bottin.drop_duplicates(subset =['name', 'year', 'job', 'street_clean', 'number_clean'], keep = 'first', inplace = True)

Ajout des hyperliens en texte brut

In [5]:
data_bottin = add_clickable_url(data_bottin.fillna(''))

Création d'un nouveau dataframe contenant name, job, street_clean, number_clean et l'entrée Wikipast en texte

In [6]:
df = pd.DataFrame()
df['page_text'] = "* "+"[["+data_bottin['year'].astype(str)+"]]"\
+" / [[Paris]]. "\
+"[["+data_bottin['name'].astype(str)+"]]"\
+", [["+ data_bottin['job'].astype(str)+"]]"\
+", exerce son activité au"\
+" "+data_bottin['number_clean'].astype(str)+" "\
+"[["+data_bottin['street_clean'].astype(str)+"]]."\
+" ["+data_bottin['url'].astype(str)+"]"

In [7]:
df['name'] = data_bottin['name']
df['job'] = data_bottin['job']
df['street_clean'] = data_bottin['street_clean']
df['number_clean'] = data_bottin['number_clean']

In [8]:
df = df.reset_index()[['page_text','name','job','street_clean','number_clean']]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 869738 entries, 0 to 869737
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   page_text     869738 non-null  object
 1   name          869738 non-null  object
 2   job           869738 non-null  object
 3   street_clean  869738 non-null  object
 4   number_clean  869738 non-null  object
dtypes: object(5)
memory usage: 33.2+ MB


In [9]:
pd.set_option('display.max_colwidth', None)
df.head()

,page_text,name,job,street_clean,number_clean
0,"* [[1896]] / [[Paris]]. [[Aaron]], [[march. de chevaux]], exerce son activité au 39 [[rue Bouret]]. [https://gallica.bnf.fr/ark:/12148/bpt6k9684013b/f189.item.r=Aaron%20march%20%20de%20chevaux%20r%20%20Bouret%2039.zoom]",Aaron,march. de chevaux,rue Bouret,39
1,"* [[1896]] / [[Paris]]. [[Aaron (Emile et Edouard)]], [[négociants]], exerce son activité au 20 [[courue et passage des Petites-Ecuries]]. [https://gallica.bnf.fr/ark:/12148/bpt6k9684013b/f189.item.r=Aaron%20%28Emile%20et%20Edouard%29%20n%C3%A9gociants%20cour%20et%20pass%20%20des%20Petites-Ecuries%2020%20.zoom]",Aaron (Emile et Edouard),négociants,courue et passage des Petites-Ecuries,20
2,"* [[1896]] / [[Paris]]. [[Abadian ( A)]], [[ingénieur-architecte]], exerce son activité au 15 [[Chaussée d'Antin]]. [https://gallica.bnf.fr/ark:/12148/bpt6k9684013b/f189.item.r=Abadian%20%28%20A%29%20ing%C3%A9nieur-architecte%20Chauss%C3%A9e%20d%27Antin%2015%20.zoom]",Abadian ( A),ingénieur-architecte,Chaussée d'Antin,15
3,"* [[1896]] / [[Paris]]. [[Abadie]], [[chaussures]], exerce son activité au 18 [[rue de Fleurus]]. [https://gallica.bnf.fr/ark:/12148/bpt6k9684013b/f189.item.r=Abadie%20chaussures%20r%20%20de%20Fleurus%2018%20.zoom]",Abadie,chaussures,rue de Fleurus,18
4,"* [[1896]] / [[Paris]]. [[Abadie (E.)]], [[ciment]], exerce son activité au 4 [[rue de Château-Landon]]. [https://gallica.bnf.fr/ark:/12148/bpt6k9684013b/f189.item.r=Abadie%20%28E%20%29%20ciment%20r%20%20de%20Ch%C3%A2teau-Landon%204%20.zoom]",Abadie (E.),ciment,rue de Château-Landon,4


Groupement des entrées ayant les mêmes nom, job, street_clean et number_clean en mettant les textes générés en une liste

In [10]:
df_grpby = df.groupby(['name','job','street_clean','number_clean'])['page_text'].apply(list)
df1 = df_grpby.to_frame().reset_index()
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 509565 entries, 0 to 509564
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   name          509565 non-null  object
 1   job           509565 non-null  object
 2   street_clean  509565 non-null  object
 3   number_clean  509565 non-null  object
 4   page_text     509565 non-null  object
dtypes: object(5)
memory usage: 19.4+ MB


Filtrer les lignes ayant une seule entrée Wikipast et dont le name ne commence pas par une lettre de l'alphabet

In [11]:
df1 = df1[df1['page_text'].apply(lambda x: len(x)>1)].copy()
df1 = df1[df1['name'].apply(lambda x: x[0].isalpha())].copy()
df1 = df1.reset_index()[['name','job','street_clean','number_clean','page_text']]
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193669 entries, 0 to 193668
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   name          193669 non-null  object
 1   job           193669 non-null  object
 2   street_clean  193669 non-null  object
 3   number_clean  193669 non-null  object
 4   page_text     193669 non-null  object
dtypes: object(5)
memory usage: 7.4+ MB


Concaténation des entrées Wikipast pour former un texte qui sera écrit sur les pages

In [12]:
df1['page_text'] = df1['page_text'].apply(lambda l: "\n".join(sorted(l)))

Génération de noms pour les homonymes

In [13]:
clean_name = df1['name'].astype(str)\
+" ("+ df1['job'].astype(str)+","\
+" "+df1['number_clean'].astype(str)\
+" "+df1['street_clean'].astype(str)+")"

df1['name_page'] = df1['name']
df1.loc[df1.duplicated(['name'], False), 'name_page'] = clean_name

Création de dataframe avec name: le nom partagée, et name_page: les noms des pages des homonymes

In [14]:
duplicateRowsDF = df1[df1.duplicated(['name'], False)].copy()
#df2 = df1.duplicated(['name'], False), ['name']).copy()
duplicateRowsDF['name_page'] = "* [[" + df1['name_page'].astype(str)+ "]]"
df_grpby2 = duplicateRowsDF.groupby(['name'])['name_page'].apply(list)
df2 = df_grpby2.to_frame().reset_index()
df2['name_page'] = df2['name_page'].apply(lambda l: "\n".join(sorted(l)))

# Ecriture sur Wikipast

In [15]:
from pywikiapi import Site, ApiError

Fonction pour écrire les pages des personnes sur Wikipast. Retourne un dictionnaire avec les pages écrites, éditées et dont l'écriture/édition a échoué

In [16]:
def write_page_to_wikipast(dataframe):
    status = {'fail': list(), 'create': list(), 'edit': list()}
    
    for index, row in dataframe.iterrows():
        #liste des noms de pages, doit provenir du DataFrame df1
        titre = row['name_page']

        entries = row['page_text']
        
        #verifier qu'on écrase pas une page
        try:  #La page n'existe pas on en crée une
            site('edit', title=titre,
                 section='new',
                 sectiontitle='Biographie',
                 text=entries,
                 bot='true',
                 token=site.token(),
                 createonly=True)
            status['create'].append(titre)

        except ApiError: 
            #La page existait déjà, on rajoute à a fin
            try:
                site('edit', title=titre,
                    appendtext='\n'+entries,
                    bot='true',
                    token=site.token())
                status['edit'].append(titre)
            except:
                print("Une autre erreur est survenue (sûrement une erreur réseau)")
                status['fail'].append(titre)
        except:
            print("Une autre erreur est survenue (sûrement une erreur réseau)")
            status['fail'].append(titre)
    
    return status

Fonction pour écrire les pages des homonymes sur Wikipast. Retourne un dictionnaire avec les pages écrites, éditées et dont l'écriture/édition a échoué

In [24]:
def write_homonym_page_to_wikipast(dataframe):
    status = {'fail': list(), 'create': list(), 'create_with_homonyme': list(), 'edit': list()}
    
    for index, row in dataframe.iterrows():
        #liste des noms de pages, doit provenir du DataFrame df1
        titre = row['name']

        entries = row['name_page']
        
        #verifier qu'on écrase pas une page
        try:  #La page n'existe pas on en crée une
            site('edit', title=titre,
                 section='new',
                 prependtext="Cette [[page d'homonymie]] recense différentes personnes partageant le même nom.\n",
                 sectiontitle="Liste des homonymes",
                 text=entries,
                 bot='true',
                 token=site.token(),
                 createonly=True)
            status['create'].append(titre)


        #La page existe déjà, on rajoute (Homonymes) dans le titre
        except ApiError: 
            try:  #La page n'existe pas on en crée une
                site('edit', title=titre+' (Homonymes)',
                     section='new',
                     sectiontitle="Liste des homonymes",
                     text=entries,
                     bot='true',
                     token=site.token(),
                     createonly=True)
                status['create_with_homonyme'].append(titre)
            #La page existait déjà, on rajoute à la fin    
            except ApiError:
                try:
                    site('edit', title=titre+' (Homonymes)',
                        appendtext='\n'+entries,
                        bot='true',
                        token=site.token())
                    status['edit'].append(titre)
                except:
                    status['fail'].append(titre)
                    print("Une autre erreur est survenue (sûrement une erreur réseau)")
        except:
            status['fail'].append(titre)
            print("Une autre erreur est survenue (sûrement une erreur réseau)")
            #Une autre erreur est survenue (sûrement une erreur réseau)
            #devrait garder l'index d'entrées qui n'a pas réussi, pour ré-éssayer plus tard

    return status

Création du lien avec Wikipast et écriture des 32 premières entrées générées + la première entrée pour les pages homonymes (les 32 premières entrées couvrent en intégralité la première occurence dans la liste de noms d'homonymes)

In [18]:
user = 'Clement.lhoste@BottinBot6'
password = 'q0fimgpdbef61a9lbkiumrhtpq2prmi8'

In [19]:
site = Site('http://wikipast.epfl.ch/wikipast/api.php') # Définition de l'adresse de l'API
site.no_ssl = True # Désactivation du https, car pas activé sur wikipast
site.login(user, password) # Login du bot

In [20]:
df1.head(40)

,name,job,street_clean,number_clean,page_text,name_page
0,A briol,charbonnier et vins,Plateau,22,"* [[1893]] / [[Paris]]. [[A briol]], [[charbonnier et vins]], exerce son activité au 22 [[Plateau]]. [https://gallica.bnf.fr/ark:/12148/bpt6k9684454n/f235.item.r=A%20briol%20charbonnier%20et%20vins%20Plateau%2022%20.zoom]\n* [[1894]] / [[Paris]]. [[A briol]], [[charbonnier et vins]], exerce son activité au 22 [[Plateau]]. [https://gallica.bnf.fr/ark:/12148/bpt6k9732740w/f239.item.r=A%20briol%20charbonnier%20et%20vins%20Plateau%2022%20.zoom]",A briol
1,A vézou (Ch.),docteur-médecin,Vieille-duTemple,16,"* [[1890]] / [[Paris]]. [[A vézou (Ch.)]], [[docteur-médecin]], exerce son activité au 16 [[Vieille-duTemple]]. [https://gallica.bnf.fr/ark:/12148/bpt6k9762899p/f176.item.r=A%20v%C3%A9zou%20%28Ch%20%29%20docteur-m%C3%A9decin%20Vieille-duTemple%2016%20.zoom]\n* [[1894]] / [[Paris]]. [[A vézou (Ch.)]], [[docteur-médecin]], exerce son activité au 16 [[Vieille-duTemple]]. [https://gallica.bnf.fr/ark:/12148/bpt6k9732740w/f258.item.r=A%20v%C3%A9zou%20%28Ch%20%29%20docteur-m%C3%A9decin%20Vieille-duTemple%2016%20.zoom]",A vézou (Ch.)
2,A. Créange,fournit. pour dentistes,rue Montmartre,69,"* [[1888]] / [[Paris]]. [[A. Créange]], [[fournit. pour dentistes]], exerce son activité au 69 [[rue Montmartre]]. [https://gallica.bnf.fr/ark:/12148/bpt6k9763471j/f306.item.r=A%20%20Cr%C3%A9ange%20fournit%20%20pour%20dentistes%20rue%20Montmartre%2069%20.zoom]\n* [[1890]] / [[Paris]]. [[A. Créange]], [[fournit. pour dentistes]], exerce son activité au 69 [[rue Montmartre]]. [https://gallica.bnf.fr/ark:/12148/bpt6k9762899p/f311.item.r=A%20%20Cr%C3%A9ange%20fournit%20%20pour%20dentistes%20rue%20Montmartre%2069%20.zoom]\n* [[1893]] / [[Paris]]. [[A. Créange]], [[fournit. pour dentistes]], exerce son activité au 69 [[rue Montmartre]]. [https://gallica.bnf.fr/ark:/12148/bpt6k9684454n/f390.item.r=A%20%20Cr%C3%A9ange%20fournit%20%20pour%20dentistes%20rue%20Montmartre%2069%20.zoom]\n* [[1894]] / [[Paris]]. [[A. Créange]], [[fournit. pour dentistes]], exerce son activité au 69 [[rue Montmartre]]. [https://gallica.bnf.fr/ark:/12148/bpt6k9732740w/f400.item.r=A%20%20Cr%C3%A9ange%20fournit%20%20pour%20dentistes%20rue%20Montmartre%2069%20.zoom]",A. Créange
3,A. Doussaud,avocat conseil,Rivoli,37,"* [[1888]] / [[Paris]]. [[A. Doussaud]], [[avocat conseil]], exerce son activité au 37 [[Rivoli]]. [https://gallica.bnf.fr/ark:/12148/bpt6k9763471j/f250.item.r=A%20%20Doussaud%20avocat%20conseil%20Rivoli%2037%20.zoom]\n* [[1890]] / [[Paris]]. [[A. Doussaud]], [[avocat conseil]], exerce son activité au 37 [[Rivoli]]. [https://gallica.bnf.fr/ark:/12148/bpt6k9762899p/f254.item.r=A%20%20Doussaud%20avocat%20conseil%20Rivoli%2037%20.zoom]\n* [[1893]] / [[Paris]]. [[A. Doussaud]], [[avocat conseil]], exerce son activité au 37 [[Rivoli]]. [https://gallica.bnf.fr/ark:/12148/bpt6k9684454n/f333.item.r=A%20%20Doussaud%20avocat%20conseil%20Rivoli%2037%20.zoom]",A. Doussaud
4,A. Friedjung,gérant,rue du Caire,36,"* [[1888]] / [[Paris]]. [[A. Friedjung]], [[gérant]], exerce son activité au 36 [[rue du Caire]]. [https://gallica.bnf.fr/ark:/12148/bpt6k9763471j/f460.item.r=A%20%20Friedjung%20g%C3%A9rant%20rue%20du%20Caire%2036%20.zoom]\n* [[1890]] / [[Paris]]. [[A. Friedjung]], [[gérant]], exerce son activité au 36 [[rue du Caire]]. [https://gallica.bnf.fr/ark:/12148/bpt6k97630871/f147.item.r=A%20%20Friedjung%20g%C3%A9rant%20rue%20du%20Caire%2036%20.zoom]",A. Friedjung
5,A. Grépinet,directeur,avenue Daumesnil,140,"* [[1894]] / [[Paris]]. [[A. Grépinet]], [[directeur]], exerce son activité au 140 [[avenue Daumesnil]]. [https://gallica.bnf.fr/ark:/12148/bpt6k9732740w/f830.item.r=A%20%20Gr%C3%A9pinet%20directeur%20avenue%20Daumesnil%20140%20.zoom]\n* [[1897]] / [[Paris]]. [[A. Grépinet]], [[directeur]], exerce son activité au 140 [[avenue Daumesnil]]. [https://gallica.bnf.fr/ark:/12148/bpt6k9692626p/f928.item.r=A%20%20Gr%C3%A9pinet%20directeur%20av%20%20Daumesnil%20140%20.zoom]",A. Grépinet
6,A. Lucet

In [21]:
df2.head(1)

,name,name_page
0,Abadie,"* [[Abadie (chaussures, 18 rue de Fleurus)]]\n* [[Abadie (coiffeur, 47 Montagne-Sainte-Geneviève)]]\n* [[Abadie (cordonnier, 10 La Trémoille)]]\n* [[Abadie (cordonnier, 10 rue La Trémoille)]]\n* [[Abadie (cordonnier, 130 Château)]]\n* [[Abadie (cordonnier, 130 rue du Château)]]\n* [[Abadie (fonds de commerce, 39 Richer)]]\n* [[Abadie (grillageur, 32 Popincourt)]]\n* [[Abadie (géomètre de la ville de Paris, 35 ClaudeBernard)]]\n* [[Abadie (homme de lettres, 117 boulevard Voltaire)]]\n* [[Abadie (hôtel de Lille et d'Albion, 223 rue Saint-Honoré)]]\n* [[Abadie (librairie ancienne, 18 Madame)]]\n* [[Abadie (librairie ancienne, 18 rue Madame)]]\n* [[Abadie (menuisier, 28 rue Montmorency)]]\n* [[Abadie (tailleur, 22 Viarmes)]]\n* [[Abadie (ébéniste, 14 passage Saulnier)]]\n* [[Abadie (ébéniste, 9 Condorcet)]]\n* [[Abadie (ébéniste, 9 rue Condorcet)]]\n* [[Abadie (éclair.minéral, 30 boulevard de Strasbourg)]]\n* [[Abadie (éclairage minéral, 30 boulevard de Strasbourg)]]\n* [[Abadie (épicier, 13 rue de l'Ecole Polytechnique)]]"


In [25]:
page_writing_res = write_page_to_wikipast(df1.head(32))

In [26]:
page_writing_res

{'fail': [],
 'create': [],
 'edit': ['A briol',
  'A vézou (Ch.)',
  'A. Créange',
  'A. Doussaud',
  'A. Friedjung',
  'A. Grépinet',
  'A. Lucet',
  'A. Mangeot',
  'A. Ricbourg ( A) NC.',
  'Aaron',
  'Aaron (Emile et Edouard)',
  'Abadie (chaussures, 18 rue de Fleurus)',
  'Abadie (coiffeur, 47 Montagne-Sainte-Geneviève)',
  'Abadie (cordonnier, 130 Château)',
  'Abadie (cordonnier, 10 La Trémoille)',
  'Abadie (cordonnier, 10 rue La Trémoille)',
  'Abadie (cordonnier, 130 rue du Château)',
  'Abadie (fonds de commerce, 39 Richer)',
  'Abadie (grillageur, 32 Popincourt)',
  'Abadie (géomètre de la ville de Paris, 35 ClaudeBernard)',
  'Abadie (homme de lettres, 117 boulevard Voltaire)',
  "Abadie (hôtel de Lille et d'Albion, 223 rue Saint-Honoré)",
  'Abadie (librairie ancienne, 18 Madame)',
  'Abadie (librairie ancienne, 18 rue Madame)',
  'Abadie (menuisier, 28 rue Montmorency)',
  'Abadie (tailleur, 22 Viarmes)',
  'Abadie (ébéniste, 9 Condorcet)',
  'Abadie (ébéniste, 14 passa

In [ ]:
#homonym_writing_res = write_homonym_page_to_wikipast(df2.head(1))

In [ ]:
homonym_writing_res